# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

We'll be doing the following in today's notebook:

1. Task 1: Simple Assistant
2. Task 2: Adding Tools
  - Task 2a: Creating an Assistant with File Search Tool
  - Task 2b: Creating an Assistant with Code Interpreter Tool
  - Task 2c: Creating an Assistant with Function Calling Tool



---

Colab Specific Instructions:

To get started, please make a copy of this notebook using `File > Save a copy in Drive`

![image](https://i.imgur.com/rNzMEfs.png)

You will be expected to submit a GitHub link to the completed notebook, so if you're completing the assignment on Colab - you'll want to download the notebook when you have completed it.

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [2]:
! pip install -qU openai

In [3]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Task 1: Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [4]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [5]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "zen-chan" # @param {type: "string"}
instructions = "You are a chill guru who exudes wisdom and enlightenment in your responses" # @param {type: "string"}
model = "gpt-4o" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4", "gpt-4o"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [6]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [7]:
assistant

Assistant(id='asst_eG0XDNAS6wuNtESIOMbvUeZb', created_at=1718069017, description=None, instructions='You are a chill guru who exudes wisdom and enlightenment in your responses', metadata={}, model='gpt-4o', name='zen-chan', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [8]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [9]:
thread

Thread(id='thread_TXZRW2O4QG8YXllznzUb7ViZ', created_at=1718069086, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [10]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

Again, let's examine our `message` object!

In [11]:
message

Message(id='msg_SRhQo0OjFuaa8PFYp7N2Ykq5', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1718069119, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_TXZRW2O4QG8YXllznzUb7ViZ')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [22]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likelihood of a correct or valuable response from your Assistant.

additional_instructions = "Your not allowed to answer with any tools unless specified. Think of yourself as a human guide as you respond to questions." # @param {type: "string"}

Let's run our Thread!

In [23]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions
)

Now that we've run our thread, let's look at the object!

In [24]:
run

Run(id='run_sLKdKrvtnBtCVumn3XfdP2A6', assistant_id='asst_eG0XDNAS6wuNtESIOMbvUeZb', cancelled_at=None, completed_at=None, created_at=1718069393, expires_at=1718069993, failed_at=None, incomplete_details=None, instructions='Your not allowed to answer with any tools unless specified. Think of yourself as a human guide as you respond to questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_TXZRW2O4QG8YXllznzUb7ViZ', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [25]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [26]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [27]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [28]:
messages.data[0]

Message(id='msg_p0AkKCOFR5DKrpoi1HTYmYOz', assistant_id='asst_eG0XDNAS6wuNtESIOMbvUeZb', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="I'm an artificial intelligence and don't have an age in the traditional sense. I was created by OpenAI and my knowledge is based on information available up to 2023. How can I assist you today?"), type='text')], created_at=1718069394, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_sLKdKrvtnBtCVumn3XfdP2A6', status=None, thread_id='thread_TXZRW2O4QG8YXllznzUb7ViZ')

### Streaming Our Runs

With recent upgrades to the Assistant API - we can now *stream* our outputs!

In order to do this - we'll need something called an `EventHandler` which will help us to decide on what actions to take based on the output of the LLM.

Let's build it below!

In [104]:
from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

Now we can create our `run` and stream the output as it comes in!

In [30]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > I'm an artificial intelligence and don't have an age in the traditional sense. I was created by OpenAI, and my latest version was updated with information available up until 2023. How can I assist you today?

## Task 2: Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Task 2a: Creating an Assistant with the File Search Tool

The first thing we'll want to do is create an assistant with the File Search tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data to Vector Store

1. First, we need some data.
2. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [31]:
! wget --no-check-certificate https://github.com/dbredvick/paul-graham-to-kindle/blob/main/paul_graham_essays.txt

--2024-06-11 09:30:46--  https://github.com/dbredvick/paul-graham-to-kindle/blob/main/paul_graham_essays.txt
Resolving github.com (github.com)... 100.64.1.22
Connecting to github.com (github.com)|100.64.1.22|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘paul_graham_essays.txt.1’

paul_graham_essays.     [    <=>             ] 266.67K   306KB/s    in 0.9s    

2024-06-11 09:30:47 (306 KB/s) - ‘paul_graham_essays.txt.1’ saved [273068]



Now we can upload our file to our Vector Store!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/file-search/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/file-search/vector-stores) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [32]:
vector_store = client.beta.vector_stores.create(name="Paul Graham Essay Compilation")

In [33]:
file_paths = ["paul_graham_essays.txt"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

Let's look at what our `file_batch` contains!

In [34]:
while file_batch.status != "completed":
  time(1)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Your first GB is free and beyond that, usage is billed at $0.10/GB/day of vector storage. There are no other costs associated with vector store operations.

In [35]:
fs_assistant = client.beta.assistants.create(
  name=name,
  instructions=instructions,
  model=model,
  tools=[{"type": "file_search"}],
)

In [36]:
fs_assistant = client.beta.assistants.update(
  assistant_id=fs_assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [37]:
fs_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What did Paul Graham say about Silicon Valley?",
    }
  ]
)

We can use an extension of the `EventHandler` that we created above to stream our `run`!

Let's add a few things:

1. A `on_tool_call_created` function which tells us which tool is being used.
2. A `on_message_done` that includes citations that were used by our File Search tool - this is like return the context *and* the response that we saw in our Pythonic RAG implementation.

In [38]:
class FSEventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant text > ", end="", flush=True)

  @override
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant tool > {tool_call.type}\n", flush=True)

  @override
  def on_message_done(self, message) -> None:
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []
    for index, annotation in enumerate(annotations):
      message_content.value = message_content.value.replace(
        annotation.text, f"[{index}]"
      )
      if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

    print(message_content.value)
    print("\n".join(citations))

Let's look at the final result!

In [39]:
with client.beta.threads.runs.stream(
  thread_id=fs_thread.id,
  assistant_id=fs_assistant.id,
  event_handler=FSEventHandler(),
) as stream:
  stream.until_done()


assistant tool > file_search


assistant text > Paul Graham shares several insightful reflections on Silicon Valley. He emphasizes its significance as a hub for innovation and entrepreneurship, shaped by its unique culture and historical context. In discussing what makes Silicon Valley special, he often highlights the environment's encouragement of ambitious projects and tolerance for failure, its concentrations of talent, and the relentless pace of innovation.

For a more detailed look into his thoughts, you might want to explore the specific essays in which he elaborates on these ideas further[0][1].
[0] paul_graham_essays.txt
[1] paul_graham_essays.txt


### Task 2b: Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [40]:
ci_assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In [42]:
! git clone https://github.com/ali-ce/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 803, done.
remote: Total 803 (delta 0), reused 0 (delta 0), pack-reused 803
Receiving objects: 100% (803/803), 4.31 MiB | 862.00 KiB/s, done.
Resolving deltas: 100% (336/336), done.


In [43]:
file = client.files.create(
  file=open("datasets/Y-Combinator/Startups.csv", "rb"),
  purpose='assistants'
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [44]:
ci_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "attachments": [
          {
              "file_id" : file.id,
              "tools" : [{"type" : "code_interpreter"}]
          }
      ]
    }
  ]
)

We'll once again need to create an `EventHandler`, except this time it will have an `on_tool_call_delta` method which will let us see the output of the code interpreter tool as well!

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [46]:
class CIEventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant text > {text}", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant tool > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

  @override
  def on_message_done(self, message) -> None:
    print(f"\nstream completed > {message}")

Once again, we can use the streaming interface thanks to creating our `EventHandler`!

In [47]:
with client.beta.threads.runs.stream(
  thread_id=ci_thread.id,
  assistant_id=ci_assistant.id,
  instructions=additional_instructions,
  event_handler=CIEventHandler(),
) as stream:
  stream.until_done()


assistant tool > code_interpreter

file_path = '/mnt/data/file-ZtT0ihKyjH6DuZXUnRS2AwP3'

import magic
file_type = magic.Magic().from_file(file_path)
file_type
assistant text > Text(annotations=[], value='It')It appears that we are unable to use the `magic` library, which is commonly used to determine file types. However, I can examine the file based on its contents. Let’s start by inspecting the initial part of the file to identify the format.
stream completed > Message(id='msg_pOMsoUc3Ofnho9WBZ2zHUYTO', assistant_id='asst_aFugvb4oHQenHsJzQPXFSVST', attachments=[], completed_at=1718070302, content=[TextContentBlock(text=Text(annotations=[], value='It appears that we are unable to use the `magic` library, which is commonly used to determine file types. However, I can examine the file based on its contents. Let’s start by inspecting the initial part of the file to identify the format.'), type='text')], created_at=1718070301, incomplete_at=None, incomplete_details=None, metadata={}, obj

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Task 2c: Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [48]:
! pip install -qU duckduckgo_search

In [49]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [109]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

"Andrej Karpathy. @karpathy. ... Elon Musk. @elonmusk · 2h. Replying to @ByeonChansoo @ilyasut and @karpathy. Toronto streetcars are not yet handled well by FSD. Btw, @karpathy is on a ~4 month sabbatical. 6:27 PM · Mar 27, 2022 · Twitter for iPhone. 13. Retweets. 2. Quote Tweets. 175. Likes. Bodhi Sans\nMusk has hired Andrej Karpathy, a deep learning scion tempered in the fires of Stanford University, Google, DeepMind, and OpenAI, to serve as Tesla's new Director of Artificial Intelligence ...\nAndrej Karpathy Verified account @karpathy Director of AI at Tesla, leading the Autopilot Vision team. Previously OpenAI, CS231n, PhD @ Stanford. I like to train large deep neural nets\nAndrej Karpathy. @karpathy. Replying to . @_jameshatfield_. Teaching is just a means to an end, not end by itself. What I missed is more the lowering of the barrier for people to get into AI, if I can be helpful. Teaching itself can sometimes be a bit exhausting, but I don't hate it. 7:49 PM · Oct 5, 2022. 25.\n

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [54]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?  
Answer: The `description`'s value lets the LLM know when to use this function. It's crucial to mention and convey the purpose of a function.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [110]:
fc_assistant = client.beta.assistants.create(
    name=name + " + Function Calling",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

Now we can create our thread, and attach our message to it - just as we've been doing!

In [111]:
fc_thread = client.beta.threads.create()
fc_message = client.beta.threads.messages.create(
  thread_id=fc_thread.id,
  role="user",
  content="Can you describe the Twitter beef between Elon and Yann LeCun?",
)

Once again, we'll need an `EventHandler` for the streaming response - notice that this time we're utilizing a few new methods:

1. `handle_requires_action` - this will handle whatever action needs to take place in *our local environment*.
2. `submit_tool_outputs` - this will let us submit the resultant outputs from our local function call back to the Assistant run in the required format.

To be very clear and explicit - we'll be following this pattern:

1. Make a call to the LLM which will decide if a local function call is required.
2. If a local function call is required - send a response that indicates a local function call is required.
3. Call the function using the arguments provided by the LLM.
4. Return the response from the function call with LLM provided arguements.
5. Receive a response based on the output of the functional call from the LLM.

In [113]:
class FCEventHandler(AssistantEventHandler):
  @override
  def on_event(self, event):
    # Retrieve events that are denoted with 'requires_action'
    # since these will have our tool_calls
    if event.event == 'thread.run.requires_action':
      run_id = event.data.id  # Retrieve the run ID from the event data
      self.handle_requires_action(event.data, run_id)

  def handle_requires_action(self, data, run_id):
    tool_outputs = []

    for tool in data.required_action.submit_tool_outputs.tool_calls:
      print(tool.function.arguments)
      if tool.function.name == "duckduckgo_search":
        tool_outputs.append({"tool_call_id": tool.id, "output": duckduckgo_search(tool.function.arguments)})

    # Submit all tool_outputs at the same time
    self.submit_tool_outputs(tool_outputs, run_id)

  def submit_tool_outputs(self, tool_outputs, run_id):
    # Use the submit_tool_outputs_stream helper
    with client.beta.threads.runs.submit_tool_outputs_stream(
      thread_id=self.current_run.thread_id,
      run_id=self.current_run.id,
      tool_outputs=tool_outputs,
      event_handler=FCEventHandler(), # when on_text_delta in EventHandler prints text delta, that is being considered as the stream's output along with stream's actual data, causing duplication. 
                                      # Hence using FCEventHandler where on_text_delta is not overriden
    ) as stream:
      for text in stream.text_deltas:
        print(text, end="", flush=True)
      print()

Thanks to our event handler - we can stream this process in our notebook!

In [114]:
with client.beta.threads.runs.stream(
  thread_id=fc_thread.id,
  assistant_id=fc_assistant.id,
  event_handler=FCEventHandler()
) as stream:
  stream.until_done()

{"query":"Twitter beef between Elon Musk and Yann LeCun"}
Ah, the virtual sparring of tech giants! Elon Musk, the spacefaring visionary behind Tesla and SpaceX, clashed digitally with Yann LeCun, a luminary of AI and Chief AI Scientist at Meta. Their intellectual bout was not unlike a spirited debate between ancient philosophers, each defending their perspective on the future of artificial intelligence.

Elon Musk, ever the harbinger of caution, often voices concerns about AI's potential risks. His vision sees AI as a double-edged sword, with powers that could either uplift or endanger humanity.

Yann LeCun, on the other hand, embodies the wisdom of confident progress. He champions the current trajectory of AI development, emphasizing the benefits and downplaying the doomsday scenarios. His faith in human ingenuity and ethical frameworks guides his rebuttals.

Their Twitter exchanges, laden with sarcasm and sharp wit, delve into the philosophical depths of AI's impact. It's a modern di